In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/sms-spam-collection-dataset/spam.csv")
df.head()

In [ ]:
df.info()

In [ ]:
from_unnamed_2 = df[~df["Unnamed: 2"].isna()][["v1","Unnamed: 2"]]
from_unnamed_3 = df[~df["Unnamed: 3"].isna()][["v1","Unnamed: 3"]]
from_unnamed_4 = df[~df["Unnamed: 4"].isna()][["v1","Unnamed: 4"]]
column_names = ["label", "text"]
from_unnamed_2.columns = from_unnamed_3.columns = from_unnamed_4.columns = column_names
from_unnamed_4

In [ ]:
df.drop(columns=["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], inplace=True)
df.columns = column_names
df.head()

In [ ]:
df.shape

In [ ]:
df = df.append([from_unnamed_2, from_unnamed_3, from_unnamed_4], ignore_index=True)
df.shape

In [ ]:
df.tail()

# Analyze data and classes a bit

In [ ]:
import seaborn as sns

In [ ]:
df["label"].value_counts()

In [ ]:
sns.countplot(x=df["label"])

Classes are imbalanced, must use smart metrics for evaluation!

In [ ]:
import re
from nltk.corpus import stopwords

def preprocess(text):
    new_text = re.sub(r"[^0-9a-zA-Z]", " ", text)
    new_text = re.sub(r"\s+", " ", text)
    new_text = [word for word in text.split(" ") if word not in stopwords.words("english")]
    new_text = " ".join(new_text)
    return new_text

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier

In [ ]:
df["spam"] = df["label"].apply(lambda t: 1 if t=="spam" else 0)
df.head()

In [ ]:
X = df["text"]
y = df["spam"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
pipe = Pipeline([("vectorizer", TfidfVectorizer(preprocessor=preprocess)), ("classifier", LinearSVC())])
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

In [ ]:
from sklearn.metrics import classification_report, plot_confusion_matrix

In [ ]:
plot_confusion_matrix(estimator=pipe, X=X_test, y_true=y_test)

In [ ]:
print(classification_report(y_true=y_test, y_pred=pipe.predict(X_test), target_names=["ham", "spam"]))